<a href="https://colab.research.google.com/github/haru1489248/nlp-100-nock/blob/main/ch09/section_83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install -U transformers torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 20.6 MB/s eta 0:00:00


In [22]:
import torch
from torchmetrics.functional import pairwise_cosine_similarity
from transformers import AutoModel, AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

# Dropoutなどの学習時専用挙動を無効化する
model.eval()

In [24]:
sentences = [
    "The movie was full of fun.",
    "The movie was full of excitement.",
    "The movie was full of crap.",
    "The movie was full of rubbish."
]

### tokenizerについて

tokenizer は複数文（バッチ入力）に対応している。

- sentences: 入力文の配列（`list[str]`）
- return_tensors: 出力を PyTorch などのテンソル形式に変換する指定
- padding: バッチ内で文の長さを `[PAD]` トークンで揃えるかどうか
- truncation: モデルの最大長を超える文章を自動で切り落とすかどうか


In [25]:
inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True)

In [26]:
with torch.no_grad():
  outputs = model(**inputs) # **はキーワード引数として展開するpythonの記法

### last_hiddenはどうなっているか
`last_hidden.shape = (batch_size, seq_len, hidden_size)`
- `batch_size`: 文の数（sentencesの数）
- `seq_len`: トークンの数（[CLS], 単語, [SEP], PAD 含む）
- `hidden_size`: 書くトークンの埋め込み次元（bert-baseなら768）


In [27]:
last_hidden = outputs.last_hidden_state

In [28]:
print(last_hidden)

tensor([[[ 0.2828, -0.2636,  0.1093,  ..., -0.4473,  0.3723,  0.2192],
         [ 0.1627, -0.7891, -0.0345,  ..., -0.5373,  0.7820, -0.3398],
         [ 0.9971, -0.4762,  0.3013,  ..., -0.3349,  0.1641, -0.3276],
         ...,
         [ 0.3238, -0.2024,  0.3026,  ..., -0.3850,  0.3314, -0.2486],
         [ 0.8131, -0.0029, -0.2271,  ...,  0.1727, -0.5665, -0.3684],
         [ 0.6521,  0.0869,  0.2177,  ...,  0.2843, -0.5923, -0.1937]],

        [[ 0.1435, -0.2701,  0.1080,  ..., -0.4082,  0.4033,  0.1364],
         [ 0.2259, -0.7829, -0.0506,  ..., -0.4750,  0.7534, -0.2578],
         [ 1.1483, -0.4832,  0.3087,  ..., -0.2553,  0.1238, -0.3381],
         ...,
         [ 0.4517, -0.6837,  0.2091,  ..., -0.8668,  0.5381, -0.3754],
         [ 0.6197,  0.0082, -0.1377,  ...,  0.1973, -0.3866, -0.3518],
         [ 0.3631,  0.1735,  0.4880,  ...,  0.3721, -0.4032, -0.0448]],

        [[ 0.3010,  0.1436, -0.0652,  ..., -0.2415,  0.4671,  0.1211],
         [ 0.6523, -0.5456, -0.1248,  ..., -0

### `last_hidden[:, 0, :]は何をしている？
- これはテンソルのスライス
  - 一次元 -> `':' = 全部（全ての文章を対象とする）`
  - 二次元 -> `'0' = 先頭トークン（BERTでは先頭=[CLS]）`
  - 三次元 -> `':' = 全部（ベクトルの全成分）`


In [29]:
cls_vecs = last_hidden[:, 0, :]

デフォルトで自分自身とのコサイン類似度は0になる様に設定されている
- zero_diagonal: Falseで対角線の値（自分自身とのコサイン類似度を1にする様に設定できる）デフォルトはTrue

In [32]:
print(pairwise_cosine_similarity(cls_vecs, zero_diagonal=False))

tensor([[1.0000, 0.9881, 0.9558, 0.9475],
        [0.9881, 1.0000, 0.9541, 0.9487],
        [0.9558, 0.9541, 1.0000, 0.9807],
        [0.9475, 0.9487, 0.9807, 1.0000]])
